<a href="https://colab.research.google.com/github/GouriNayak/DEMO/blob/main/Weatherforecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the required libraries


In [ ]:
import numpy as np  # linear algebra
import pandas as pd # Data processing , CSV file I/O (e.g. pd.read_csv)

Load the csv file as data frame

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/GouriNayak/DEMO/main/weatherAUS.csv")

In [ ]:
print('size of weather data frame is :',df.shape)

size of weather data frame is : (142193, 24)


Display data

In [ ]:
print(df[0:5])

         Date Location  MinTemp  ...  RainToday  RISK_MM  RainTomorrow
0  2008-12-01   Albury     13.4  ...         No      0.0            No
1  2008-12-02   Albury      7.4  ...         No      0.0            No
2  2008-12-03   Albury     12.9  ...         No      0.0            No
3  2008-12-04   Albury      9.2  ...         No      1.0            No
4  2008-12-05   Albury     17.5  ...         No      0.2            No

[5 rows x 24 columns]


Checking null values

In [ ]:
print(df.count().sort_values())

Sunshine          74377
Evaporation       81350
Cloud3pm          85099
Cloud9am          88536
Pressure9am      128179
Pressure3pm      128212
WindDir9am       132180
WindGustDir      132863
WindGustSpeed    132923
WindDir3pm       138415
Humidity3pm      138583
Temp3pm          139467
WindSpeed3pm     139563
Humidity9am      140419
RainToday        140787
Rainfall         140787
WindSpeed9am     140845
Temp9am          141289
MinTemp          141556
MaxTemp          141871
Date             142193
Location         142193
RISK_MM          142193
RainTomorrow     142193
dtype: int64


# As we can see the first 4 coloumns have less than 60% data, we can ignore these 4 colur

# we don't need the location column because we are going to find if it will rain in Australia(not lacation specific)

# we are going to drop the date column too.

# we need to remove RISK_MM because we want to predict 'RainTomorrow' and RISK_MM can leak
# we remove unwanted data
'Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','RISK_MM','Date'

In [ ]:
df = df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','RISK_MM','Date'],axis=1)

In [ ]:
print(df.shape)

(142193, 17)


# Let us get rid of all null values in df

In [ ]:
df = df.dropna(how='any')

In [ ]:
print(df.shape)

(112925, 17)
